# Date Related Types

Pydantic offers us a number of date related types.

Let's look at a few.

In [1]:
from pydantic import (
    BaseModel, 
    ConfigDict, 
    PastDatetime, 
    PastDate, 
    AwareDatetime, 
    NaiveDatetime, 
    ValidationError,
)

`PastDatetime` will restrict a field to be a valid datetime object, and also to be in the past compared to your local current time.

The docs are not exactly very forthcoming about this, but how are naive datetimes interpreted when determining if they are past or not?

Turns out they assume naive datetimes to be local, not UTC.

Let's test this out...

In [2]:
from datetime import datetime, timedelta

import pytz

local_one_hour_ago = datetime.now() - timedelta(hours=1)
local_one_hour_ago

datetime.datetime(2023, 12, 2, 10, 18, 54, 126047)

Now, according to my timezone in AZ (we have no DST), this time, in UTC, would be:

In [3]:
utc_one_hour_ago = local_one_hour_ago.astimezone(pytz.utc)

But I want this datetime to be naive, not aware, so we can see how Pydantic handles it in relation to local time.

In [4]:
utc_naive_one_hour_ago = utc_one_hour_ago.replace(tzinfo=None)
utc_naive_one_hour_ago

datetime.datetime(2023, 12, 2, 17, 18, 54, 126047)

Ok, so now let's try using `PastDatetime`.

In [5]:
class Model(BaseModel):
    dt: PastDatetime

In [6]:
m = Model(dt=local_one_hour_ago)
m

Model(dt=datetime.datetime(2023, 12, 2, 10, 18, 54, 126047))

In [7]:
m = Model(dt=utc_one_hour_ago)
m

Model(dt=datetime.datetime(2023, 12, 2, 17, 18, 54, 126047, tzinfo=<UTC>))

So, both of these work fine. 

And to see that Pydantic will interpret a naive datetime as a local time:

In [8]:
try:
    Model(dt=utc_naive_one_hour_ago)
except ValidationError as ex:
    print(ex)

1 validation error for Model
dt
  Input should be in the past [type=datetime_past, input_value=datetime.datetime(2023, 12, 2, 17, 18, 54, 126047), input_type=datetime]
    For further information visit https://errors.pydantic.dev/2.5/v/datetime_past


Let's look at some of the other datetime types.

We have the NaiveDatetime type - this basically requires the datetime passed in to be naive.

In [9]:
class Model(BaseModel):
    dt: NaiveDatetime

In [10]:
local_one_hour_ago

datetime.datetime(2023, 12, 2, 10, 18, 54, 126047)

In [11]:
Model(dt=local_one_hour_ago)

Model(dt=datetime.datetime(2023, 12, 2, 10, 18, 54, 126047))

But, if we try to pass in an aware datetime:

In [12]:
utc_one_hour_ago

datetime.datetime(2023, 12, 2, 17, 18, 54, 126047, tzinfo=<UTC>)

In [13]:
try:
    Model(dt=utc_one_hour_ago)
except ValidationError as ex:
    print(ex)

1 validation error for Model
dt
  Input should not have timezone info [type=timezone_naive, input_value=datetime.datetime(2023, 1...4, 126047, tzinfo=<UTC>), input_type=datetime]
    For further information visit https://errors.pydantic.dev/2.5/v/timezone_naive


And then we have the flip side of this, where we might require our datetime to be aware:

In [14]:
class Model(BaseModel):
    dt: AwareDatetime

In [15]:
Model(dt=utc_one_hour_ago)

Model(dt=datetime.datetime(2023, 12, 2, 17, 18, 54, 126047, tzinfo=<UTC>))

But validation will fail if we pass in a naive datetime:

In [16]:
try:
    Model(dt=local_one_hour_ago)
except ValidationError as ex:
    print(ex)

1 validation error for Model
dt
  Input should have timezone info [type=timezone_aware, input_value=datetime.datetime(2023, 12, 2, 10, 18, 54, 126047), input_type=datetime]
    For further information visit https://errors.pydantic.dev/2.5/v/timezone_aware


Very often, since datetime arithmetic is **not** especially easy, we end up writing custom validators and serializers.

For example, the recommened way to work with datetimes in our Python apps, is to convert everything to UTC (often as naive datetimes). This way arithmetic is much easier, since UTC does not use DST. And then we only convert to some localized (usually aware) datetime when "displaying" the data to our users.

For that, we may end up writing a custom validator that will deserialize any datetime into a UTC datetime, with some assumptions made as to what timezone should be assumed if the input datetime is naive, and do timezone conversion if it is aware. We'll circle back to this later.